In [33]:
# import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

def safe_find_element_text(element, by, value, default=""):
    """
        Avoids NoSuchElement Exception from selenium
    """
    try:
        return element.find_element(by, value).text
    except Exception:
        return default

def safe_find_element_attribute(element, by, value, attribute, default=""):
    """
        Avoids NoSuchElement Exception from selenium
    """
    try:
        return element.find_element(by, value).get_attribute(attribute)
    except Exception:
        return default
    
all_pages = []    

In [34]:
mdriver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [35]:
country_page = [('Ghana',2),('Kenya',4),('Malawi',3),('Nigeria',2),('Rwanda',2),('Sierra+Leone',2),('South%20Africa',66),('Tanzania',3),('Uganda',2),('Zambia',2)]
for country, page_num in country_page:
    for num in range(1,page_num):
        main_url = "https://www.mott.org/grants/?location=" + country + "&pg=" + str(num)
        # load the web page
        mdriver.get(main_url)

        # set maximum time to load the web page in seconds
        mdriver.implicitly_wait(5)

        grants = mdriver.find_element(By.CLASS_NAME, "archives2-entries").find_elements(By.CLASS_NAME, "card6")

        for grant_link in grants:
        
            link = grant_link.find_element(By.CLASS_NAME, "card6-link")
            link.click()
        
            grant_url = mdriver.current_url
            mdriver.get(grant_url)
            mdriver.implicitly_wait(2)
            grant = mdriver.find_element(By.ID, "content")

            grant_dict = {}
            
            grant_dict['grantee_name'] = safe_find_element_text(grant, By.CLASS_NAME, "insert1-subtitle")
            grant_dict['grant_description'] = safe_find_element_text(grant, By.CLASS_NAME, "insert1-title")
            try:
                site_link = grant.find_element(By.CLASS_NAME, "insert1-lead")
                grant_dict['grantee_website'] = safe_find_element_attribute(site_link, By.TAG_NAME, "a", 'href')
            except Exception:
                grant_dict['grantee_website'] = ""
            for detail in grant.find_element(By.CLASS_NAME, "list1-entries").find_elements(By.TAG_NAME, "li"):
                grant_dict[safe_find_element_text(detail, By.TAG_NAME, "strong")] = safe_find_element_text(detail, By.TAG_NAME, "span")
            
            all_pages.append(grant_dict)

            mdriver.back() # to avoid StaleElementReferenceException
            mdriver.implicitly_wait(2)

        time.sleep(5)

In [36]:
len(all_pages)

737

In [37]:
with open("mottdata.json", "w") as file:
    json.dump(all_pages, file, indent=4)